In [42]:
import os
import re
import torch
import urllib.request
from importlib.metadata import version

import warnings
warnings.filterwarnings("ignore")

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))
print("PyTorch version:", torch.__version__)

torch version: 2.6.0
tiktoken version: 0.9.0
PyTorch version: 2.6.0+cu124


# Token 


In [ ]:
if not os.path.exists("data/the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "data/the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

In [3]:
with open("data/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total de caracteres:", len(raw_text))
# 100 primeiros caracteres
print(raw_text[:99])

Total de caracteres: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


### Transformação
* Separa as palavras da pontuação.
* Remove espaços em branco.

In [7]:
text = "Olá Mundo. Isso é um teste."
result = re.split(r'([,.]|\s)', text)
result = [item for item in result if item.strip()]
print(result)

['Olá', 'Mundo', '.', 'Isso', 'é', 'um', 'teste', '.']


 - Pontuação fora do padrão

In [8]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


* Transformando o texto baixado

In [12]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))
print(preprocessed[:10])

4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius']


# Convertendo

* Construindo o vocabulário

In [13]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [14]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [16]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 20:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)


In [19]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        # Recebe um vocabulário (dicionário) que mapeia strings para inteiros.
        self.str_to_int = vocab
        # Cria um dicionário inverso que mapeia inteiros de volta para strings.
        self.int_to_str = {i: s for s, i in vocab.items()}
    
    def encode(self, text):
        """
        Converte um texto em uma lista de inteiros com base no vocabulário.
        """
        # Divide o texto em tokens usando pontuação e espaços como delimitadores.
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        # Remove espaços em branco e tokens vazios.
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        # Converte os tokens em seus respectivos IDs inteiros usando o vocabulário.
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        """
        Converte uma lista de inteiros de volta para o texto original.
        """
        # Converte os IDs de volta para strings e junta com espaços.
        text = " ".join([self.int_to_str[i] for i in ids])
        # Remove o espaço antes de certos sinais de pontuação (ex: " !", " ?" → "!", "?")
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text


- Aplicando a transformação

In [20]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know,"
Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


### Token incial e final

In [22]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

len(vocab.items())

1132

In [23]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [24]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        # Recebe um vocabulário (dicionário) que mapeia strings para inteiros.
        self.str_to_int = vocab
        # Cria um dicionário inverso que mapeia inteiros de volta para strings.
        self.int_to_str = {i: s for s, i in vocab.items()}
    
    def encode(self, text):
        """
        Converte um texto em uma lista de inteiros com base no vocabulário.
        """
        # Divide o texto em tokens usando pontuação e espaços como delimitadores.
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        # Remove espaços em branco e tokens vazios.
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        # Substitui qualquer token que não esteja no vocabulário por um token especial "<|unk|>"
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed ]
        # Converte os tokens em seus respectivos IDs inteiros usando o vocabulário.
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        """
        Converte uma lista de inteiros de volta para o texto original.
        """
        # Converte os IDs de volta para strings e junta com espaços.
        text = " ".join([self.int_to_str[i] for i in ids])
        # Remove o espaço antes de certos sinais de pontuação (ex: " !", " ?" → "!", "?")
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text


In [25]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [26]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [27]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

# BytePair encoding

In [29]:
# pip install tiktoken

In [30]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.9.0


In [31]:
tokenizer = tiktoken.get_encoding("gpt2")

In [32]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [ ]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


# Sliding window

In [36]:
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [38]:
enc_sample = enc_text[50:]

context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [39]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


* Texto completo

In [40]:
for i in range(1, context_size+1):
    context = enc_text[:i]
    desired = enc_text[i]

    print(context, "---->", desired)

[40] ----> 367
[40, 367] ----> 2885
[40, 367, 2885] ----> 1464
[40, 367, 2885, 1464] ----> 1807


In [41]:
for i in range(1, context_size+1):
    context = enc_text[:i]
    desired = enc_text[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

I ---->  H
I H ----> AD
I HAD ---->  always
I HAD always ---->  thought


In [43]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        # Listas para armazenar as sequências de entrada (input) e de saída (target)
        self.input_ids = []
        self.target_ids = []

        # Tokeniza todo o texto de uma vez, incluindo tokens especiais permitidos
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        
        # Garante que o número de tokens seja maior que o comprimento máximo exigido
        assert len(token_ids) > max_length, "Número de tokens deve ser maior que max_length + 1"

        # Utiliza uma janela deslizante (sliding window) para criar sequências sobrepostas de comprimento fixo
        for i in range(0, len(token_ids) - max_length, stride):
            # Cria a sequência de entrada (input) com tamanho max_length
            input_chunk = token_ids[i:i + max_length]
            # Cria a sequência de saída (target) deslocada em 1 posição (para previsão do próximo token)
            target_chunk = token_ids[i + 1: i + max_length + 1]
            
            # Converte as listas para tensores e armazena
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        # Retorna o número de pares (input, target) disponíveis
        return len(self.input_ids)

    def __getitem__(self, idx):
        # Retorna o par (input, target) correspondente ao índice solicitado
        return self.input_ids[idx], self.target_ids[idx]


def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    """
    Cria um DataLoader a partir de um texto cru para treinar modelos como o GPT.
    
    Parâmetros:
    - txt: texto de entrada (string) a ser tokenizado e processado
    - batch_size: tamanho do lote (quantidade de exemplos por batch)
    - max_length: tamanho máximo de cada sequência de entrada
    - stride: passo da janela deslizante (define a sobreposição entre as sequências)
    - shuffle: embaralha os dados em cada epoch
    - drop_last: descarta o último batch se ele for menor que batch_size
    - num_workers: número de subprocessos para carregar os dados (0 = usa o processo principal)
    """

    # Inicializa o tokenizador baseado no modelo gpt2
    tokenizer = tiktoken.get_encoding("gpt2")

    # Cria o dataset customizado com base no texto e no tokenizador
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Cria o DataLoader a partir do dataset, com os parâmetros fornecidos
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    # Retorna o DataLoader pronto para uso no treinamento
    return dataloader


In [44]:
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [45]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [46]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [49]:
dataloader = create_dataloader_v1(raw_text, batch_size=4, max_length=3, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885],
        [ 1807,  3619,   402],
        [10899,  2138,   257],
        [15632,   438,  2016]])

Targets:
 tensor([[ 367, 2885, 1464],
        [3619,  402,  271],
        [2138,  257, 7026],
        [ 438, 2016,  257]])


In [53]:
input_ids = torch.tensor([2, 3, 5, 1])

In [54]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [55]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [56]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [57]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


# Encoder

In [66]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [67]:
max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [68]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [69]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)
# print(token_embeddings)

torch.Size([8, 4, 256])


In [70]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [71]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [72]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
